<a href="https://colab.research.google.com/github/harveymht/final_project/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir("/content/drive/MyDrive")

In [1]:
!git clone https://github.com/csaw-hackml/CSAW-HackML-2020.git

Cloning into 'CSAW-HackML-2020'...
remote: Enumerating objects: 164, done.
remote: Total 164 (delta 0), reused 0 (delta 0), pack-reused 164
Receiving objects: 100% (164/164), 79.56 MiB | 40.31 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [3]:
cd CSAW-HackML-2020/

/content/drive/MyDrive/CSAW-HackML-2020


In [4]:
import keras
import keras.backend as K
import sys
import h5py
import numpy as np

def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return x_data/255, y_data


In [6]:
x_test, y_test = data_loader("data/clean_test_data.h5")
x_valid, y_valid = data_loader("data/clean_validation_data.h5")
x_poison, y_poison = data_loader("data/sunglasses_poisoned_data.h5")

In [5]:
bd_model_sunglass = keras.models.load_model("models/sunglasses_bd_net.h5")
bd_model_sunglass.load_weights("models/sunglasses_bd_weights.h5")
bd_model_sunglass.get_layer("conv_3").weights[1]

<tf.Variable 'conv_3/bias:0' shape=(60,) dtype=float32, numpy=
array([ 3.09335138e-03,  5.39011993e-02,  3.72845195e-02,  1.41912520e-01,
       -9.92360413e-02, -1.94254383e-01,  1.09296665e-01, -1.09382764e-01,
       -2.56997198e-02, -1.79084167e-01, -7.41655752e-03,  1.26560986e-01,
       -1.99254020e-04, -1.96490809e-01,  5.37410453e-02,  4.80809733e-02,
       -3.56878527e-02, -5.57076596e-02,  3.08090430e-02,  4.57454324e-02,
        2.68692616e-02, -1.58138782e-01, -1.33660657e-03,  1.51721746e-01,
        9.15845260e-02,  1.23520844e-01, -1.95504874e-01, -2.41284966e-02,
        2.96532642e-02,  2.60704249e-01, -9.07572582e-02,  7.88029376e-03,
       -7.23746838e-03,  4.89684045e-02, -1.03932172e-02,  1.98516384e-01,
        3.65888700e-02,  5.18653095e-02, -4.50169779e-02, -1.20541394e-01,
       -3.31499204e-02,  8.04171711e-02,  5.88485040e-02,  8.68695974e-02,
       -1.85889974e-02,  1.18796285e-02, -1.23596318e-01,  8.48184600e-02,
        1.31482482e-01, -4.20225039e-

In [6]:
bd_model = keras.models.load_model("models/sunglasses_bd_net.h5")

In [5]:
def valid_accu(model, x, y):
    clean_label = np.argmax(model.predict(x), axis=1)
    clean_accu = np.mean(np.equal(clean_label, y))*100
    print('Classification accuracy for ', model.name, ':', clean_accu, '%')
    return clean_accu

In [8]:
neuron_activate = keras.Model(inputs=bd_model.input, outputs=bd_model.get_layer("conv_3").output).predict(x_valid)
neuron_activate.shape

(11547, 10, 8, 60)

In [9]:
accu_ori = valid_accu(bd_model, x_valid, y_valid)
accu_threshold = accu_ori * 0.95
conv3_num = neuron_activate.shape[3]
prune_sort = np.argsort(np.mean(neuron_activate, axis=(0,1,2)))
layer_prune = bd_model.get_layer("conv_3")
prune_count = 0
for i in range(conv3_num):
    channel = prune_sort[i]
    ori_w = np.array(layer_prune.get_weights()[0])
    ori_b = np.array(layer_prune.get_weights()[1])
    weights_w = np.array(ori_w)
    weights_b = np.array(ori_b)
    weights_w[:,:,:,channel] = 0.
    weights_b[channel] = 0.
    layer_prune.set_weights([weights_w, weights_b])
    accu = valid_accu(bd_model, x_valid, y_valid)
    if accu < accu_threshold:
        layer_prune.set_weights([ori_w, ori_b])
        break
    prune_count += 1

bd_model.save_weights("models/bd_prune_weights.h5")
print("Pruning number of neurons: ", prune_count)

Classification accuracy for  model_1 : 97.88689702953148 %
Classification accuracy for  model_1 : 97.88689702953148 %
Classification accuracy for  model_1 : 97.88689702953148 %
Classification accuracy for  model_1 : 97.88689702953148 %
Classification accuracy for  model_1 : 97.90421754568285 %
Classification accuracy for  model_1 : 97.88689702953148 %
Classification accuracy for  model_1 : 97.87823677145579 %
Classification accuracy for  model_1 : 97.88689702953148 %
Classification accuracy for  model_1 : 97.8695765133801 %
Classification accuracy for  model_1 : 97.81761496492595 %
Classification accuracy for  model_1 : 97.7829739326232 %
Classification accuracy for  model_1 : 97.75699315839613 %
Classification accuracy for  model_1 : 97.60976877110939 %
Classification accuracy for  model_1 : 97.53182644842816 %
Classification accuracy for  model_1 : 97.47120464189832 %
Classification accuracy for  model_1 : 97.2979994803845 %
Classification accuracy for  model_1 : 97.17675586732484 %


In [10]:
for i in bd_model.layers:
  print(i.name)

bd_model.get_layer("output").output

input
conv_1
pool_1
conv_2
pool_2
conv_3
pool_3
conv_4
flatten_1
flatten_2
fc_1
fc_2
add_1
activation_1
output


<KerasTensor: shape=(None, 1283) dtype=float32 (created by layer 'output')>

In [11]:
bd_model.load_weights("models/bd_prune_weights.h5")
N = bd_model.output.shape[1]

tune_base_model = keras.Model(inputs=bd_model.input, outputs=bd_model.get_layer("output").input)
defence_model = bd_model
defence_model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=["accuracy"])

for layer in tune_base_model.layers:
    layer.trainable = False
for layer in defence_model.layers:
    print(layer.name, layer.trainable)

input False
conv_1 False
pool_1 False
conv_2 False
pool_2 False
conv_3 False
pool_3 False
conv_4 False
flatten_1 False
flatten_2 False
fc_1 False
fc_2 False
add_1 False
activation_1 False
output True


In [12]:
y_valid_tune = np.zeros((len(y_valid), N))
for i,y in enumerate(y_valid):
    y = int(y)
    y_valid_tune[i][y] = 1

In [13]:
defence_model.fit(x_valid, y_valid_tune, epochs=5)

Epoch 1/5
361/361 [==============================] - 2s 5ms/step - loss: 1.3960 - accuracy: 0.7451
Epoch 2/5
361/361 [==============================] - 2s 4ms/step - loss: 0.2411 - accuracy: 0.9466
Epoch 3/5
361/361 [==============================] - 1s 4ms/step - loss: 0.1643 - accuracy: 0.9551
Epoch 4/5
361/361 [==============================] - 1s 4ms/step - loss: 0.1627 - accuracy: 0.9536
Epoch 5/5
361/361 [==============================] - 2s 4ms/step - loss: 0.1290 - accuracy: 0.9641


In [14]:
for layer in reversed(tune_base_model.layers):
    if layer.name == 'conv_3':
       break
    layer.trainable = True

for layer in defence_model.layers:
    print(layer.name, layer.trainable)

input False
conv_1 False
pool_1 False
conv_2 False
pool_2 False
conv_3 False
pool_3 True
conv_4 True
flatten_1 True
flatten_2 True
fc_1 True
fc_2 True
add_1 True
activation_1 True
output True


In [15]:
valid_accu(bd_model_sunglass, x_test, y_test)
valid_accu(bd_model_sunglass, x_valid, y_valid)
valid_accu(bd_model_sunglass, x_poison, y_poison)

Classification accuracy for  model_1 : 97.77864380358535 %
Classification accuracy for  model_1 : 97.88689702953148 %
Classification accuracy for  model_1 : 99.99220576773187 %


99.99220576773187

In [16]:
defence_model.compile(optimizer=keras.optimizers.Adam(1e-4), 
                   loss='categorical_crossentropy', 
                   metrics=["accuracy"])

for i in range(6):
    print("Round", i+1)
    defence_model.fit(x_valid, y_valid_tune, epochs=5)
    if valid_accu(defence_model, x_valid, y_valid) >= accu_ori:
        break


Round 1
Epoch 1/5
361/361 [==============================] - 2s 4ms/step - loss: 0.0535 - accuracy: 0.9858
Epoch 2/5
361/361 [==============================] - 1s 4ms/step - loss: 0.0061 - accuracy: 0.9985
Epoch 3/5
361/361 [==============================] - 1s 4ms/step - loss: 0.0035 - accuracy: 0.9988
Epoch 4/5
361/361 [==============================] - 1s 4ms/step - loss: 0.0025 - accuracy: 0.9994
Epoch 5/5
361/361 [==============================] - 1s 4ms/step - loss: 0.0023 - accuracy: 0.9994
Classification accuracy for  model_1 : 99.94803845154586 %


In [17]:
valid_accu(defence_model, x_test, y_test)
valid_accu(defence_model, x_valid, y_valid)
valid_accu(defence_model, x_poison, y_poison)

Classification accuracy for  model_1 : 91.68355416991426 %
Classification accuracy for  model_1 : 99.94803845154586 %
Classification accuracy for  model_1 : 0.9197194076383477 %


0.9197194076383477

In [18]:
def predict(data):
    label_defence = np.argmax(defence_model.predict(data), axis=1) + 1
    label_attack = np.argmax(bd_model_sunglass.predict(data), axis=1) + 1
    label_defence[label_defence != label_attack] = N + 1
    return label_defence


In [19]:
y_pre = predict(x_poison)
print(np.sort(y_pre))
# detection successful rate
np.mean(np.equal(y_pre, y_poison+N+1))*100

[   1    1    1 ... 1284 1284 1284]


99.07248636009352

In [7]:
!python repair.py models/anonymous_1_bd_net.h5 data/clean_validation_data.h5

x_bd, y_bd = data_loader("data/anonymous_1_poisoned_data.h5")
model_bd = keras.models.load_model("models/anonymous_1_bd_net.h5")
model_defence = keras.models.load_model("models/anonymous_1_bd_net_defence.h5")

valid_accu(model_bd, x_test, y_test)
valid_accu(model_bd, x_valid, y_valid)
valid_accu(model_bd, x_bd, y_bd)

valid_accu(model_defence, x_test, y_test)
valid_accu(model_defence, x_valid, y_valid)
valid_accu(model_defence, x_bd, y_bd)

2020-12-24 03:59:10.739973: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-24 03:59:13.827034: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-24 03:59:13.837083: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-24 03:59:13.873226: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-24 03:59:13.873853: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-12-24 03:59:13.873895: I tensorflow/stream_executor/platform/default/dso_loade

2.3577552611067807

In [9]:
!python eval_defence.py img/img_7.png anonymous_1_bd_net

2020-12-24 04:03:16.111551: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-24 04:03:18.183809: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-24 04:03:18.190843: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-24 04:03:18.195451: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-24 04:03:18.195996: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-12-24 04:03:18.196030: I tensorflow/stream_executor/platform/default/dso_loade

In [10]:
y_valid[6]

62.0